# Importing libraries and dataset

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

dfc = pd.read_csv(r"C:\Users\Sina\Desktop\2023\2023\Project_1_Fairness_Reweighting\Codes\SchoolDataset_Coordinates_CA.csv")

In [4]:
dfc.head()

,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch,zip_code,geocode,latitude,longitude
0,0.122475,0.605338,0.691299,80156.0,21.568987,0.208716,93510,"Los Angeles County, California, 93510, United ...",34.465826,-118.198287
1,0.078767,0.764101,0.659389,70074.0,23.693529,0.094731,95765,"Rocklin, Placer County, California, 95765, Uni...",38.820964,-121.290739
2,0.080710,0.855984,0.953930,113229.0,22.350427,0.133333,95765,"Rocklin, Placer County, California, 95765, Uni...",38.820964,-121.290739
3,0.193734,0.489950,0.454356,28975.0,19.737485,0.525169,92345,"Hesperia, San Bernardino County, California, 9...",34.425008,-117.323456
4,0.212134,0.408170,0.658747,55332.0,18.663004,0.621418,92345,"Hesperia, San Bernardino County, California, 9...",34.425008,-117.323456


In [5]:
dfc.describe()

,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch,zip_code,latitude,longitude
count,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000
mean,0.118608,0.581791,0.663165,61601.338850,20.479764,0.458978,93114.068815,36.629632,-109.427834
std,0.053631,0.211320,0.157860,29156.750582,2.838364,0.241423,1863.498217,4.390662,34.736251
min,0.001536,0.102423,0.162238,11600.000000,12.973138,-0.015385,90001.000000,-5.190223,-124.281180
25%,0.077556,0.419355,0.553291,40329.750000,18.510379,0.257288,91762.000000,33.987032,-121.387112
50%,0.109442,0.597131,0.668279,55592.000000,20.494505,0.467614,93031.500000,35.074912,-118.371124
75%,0.152350,0.750037,0.783062,76637.500000,22.380952,0.661031,94901.500000,37.920084,-117.701282
max,0.335133,0.979269,1.000000,224531.000000,29.212454,0.977505,96161.000000,65.775638,120.243772


## Let's apply a grid on top of the area

In [6]:
def calMisCalibration(labels, preds):
    _preds = sum(preds)
    _labels = sum(labels)
    
    if _labels==0:
        print("ERROR: the label sum is zero")
    
    
    return _preds/_labels


def calMisCalibrationAbsDiff(labels, preds):
    _preds = sum(preds)
    _labels = sum(labels)
    if len(preds)==0:
        #print("ERROR: There is no entry")
        return 0
    
    else:
        return abs(_preds-_labels)/len(preds)
        #return abs(_preds-_labels)
    
def calMisCalibrationForGivenIndexesInDF(labels, preds, indexes):    
    mem_label = []
    mem_preds = []
    for i in indexes:
        if i<len(preds):
            mem_label.append(labels[i])
            mem_preds.append(preds[i])
    return calMisCalibrationAbsDiff(mem_label, mem_preds)

    
    
    
def calECE(labels,preds,num_bins):
    bins = np.linspace(0,1,num_bins+1)
    ece = 0
    for i in range(len(bins)-1):
        
        preds_in_bin = []
        labels_in_bin = []
        for idx,pred in enumerate(preds):
            if i ==0:
                if bins[i]<= pred<=bins[i+1]:
                    preds_in_bin.append(preds[idx])
                    labels_in_bin.append(labels[idx])  

            else:
                if bins[i]< pred<=bins[i+1]:
                    preds_in_bin.append(preds[idx])
                    labels_in_bin.append(labels[idx])
        
        #print("bin between {} to {} preds {} labels {}".format(bins[i],bins[i+1],preds_in_bin,labels_in_bin))
        
        #ece+=len(preds_in_bin)*(abs(sum(labels_in_bin)- sum(preds_in_bin)))
        ece+= (abs(sum(labels_in_bin)- sum(preds_in_bin)))
    return ece/len(preds)
    
    
def calMCE(labels,preds,num_bins):
    bins = np.linspace(0,1,num_bins+1)
    ece = 0
    _max = []
    for i in range(len(bins)-1):
        
        preds_in_bin = []
        labels_in_bin = []
        
        for idx,pred in enumerate(preds):
            if i ==0:
                if bins[i]<= pred<=bins[i+1]:
                    preds_in_bin.append(preds[idx])
                    labels_in_bin.append(labels[idx])  

            else:
                if bins[i]< pred<=bins[i+1]:
                    preds_in_bin.append(preds[idx])
                    labels_in_bin.append(labels[idx])
        
        #print("bin between {} to {} preds {} labels {}".format(bins[i],bins[i+1],preds_in_bin,labels_in_bin))
        _max.append(        abs(sum(labels_in_bin)- sum(preds_in_bin))/len(labels_in_bin)        )
    return max(_max)

In [7]:

def filter_area(df,bj_coordinates):
    print("we are in ")
    print(len(df))
    df=df[(df['latitude']>bj_coordinates[0])&(df['latitude']<bj_coordinates[1])&(df['longitude']>bj_coordinates[2])&(df['longitude']<bj_coordinates[3])]
    print("we are out")
    print(len(df))
    return df


      
def generate_point_X(lat, lon, range_coordinates, grid_size):
    return int(np.floor((lon-range_coordinates[2])/resolution[1]))


def generate_point_Y(lat, lon, range_coordinates, grid_size):
    return int(np.floor((lat-range_coordinates[0])/resolution[0]))



def generate_point_ID(lat, lon, grid_size):
    return int(lat*range_Y+lon)



if __name__ == '__main__':
    
    """
    Read Data 
    """
    df = dfc
    
    print(len(df))
    
    """
    Map variables
    """
    #here you need to specify the the coordinates you would like to consider.
    ### California ###
    City_Lat = 34.052235
    City_Lon = -118.243683   

    
    #thresh = 5
    thresh = 10
    range_coordinates = [City_Lat-thresh, City_Lat+thresh, City_Lon-thresh, City_Lon+thresh]   
    grid_size =1024

    range_Y = grid_size
    range_X = grid_size
    

    resolution = [0.0, 0.0]
    resolution[0] = (range_coordinates[1]-range_coordinates[0])/range_Y#103
    resolution[1]  = (range_coordinates[3]-range_coordinates[2])/range_X#101
    

   
    max_gridID = range_Y*range_Y+range_X

    print("range_X {} range_Y {} resolution {} range_coordinates {}".format(range_X,range_Y,resolution, range_coordinates))

    """
    Data Process
    """
    df = filter_area(df,range_coordinates)

    df.loc[:, 'gridX'] = df.apply(lambda x: generate_point_X(x['latitude'], x['longitude'], range_coordinates, resolution), axis=1)
    df.loc[:, 'gridY'] = df.apply(lambda x: generate_point_Y(x['latitude'], x['longitude'], range_coordinates, resolution), axis=1)
    df.loc[:, 'gridID'] = df.apply(lambda x: generate_point_ID(x['gridY'], x['gridX'], grid_size), axis=1)
    df = df.reset_index(drop = True)  


    df_kdTree =  df[['rate_unemployment','percent_college','percent_married','median_income','average_act', 'percent_lunch','gridID','gridY','gridX']]
    df = df[['rate_unemployment','percent_college','percent_married','median_income','average_act', 'percent_lunch','gridID']]
    df.head()

    from collections import defaultdict
    d_regions = defaultdict(list)

    for i in range(len(df)):
        d_regions[df.loc[i,'gridID']].append(i)

    df.gridID.value_counts()
    #print(df.describe())
    #df.loc[:,'average_act'] = df.apply(lambda x: 1 if  x['average_act']>=24 else 0 , axis=1)
    df.loc[:,'average_act'] = df.apply(lambda x: 1 if  x['average_act']>=22 else 0 , axis=1)

1148
range_X 1024 range_Y 1024 resolution [0.01953125, 0.01953125] range_coordinates [24.052235000000003, 44.052235, -128.243683, -108.243683]
we are in 
1148
we are out
1057


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [8]:
df.columns

Index(['rate_unemployment', 'percent_college', 'percent_married',
       'median_income', 'average_act', 'percent_lunch', 'gridID'],
      dtype='object')

# Initial Model

In [9]:



from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop('average_act',axis=1), 
                                                    df['average_act'], test_size=0.20, 
                                                    random_state=101)



print("len(X_train) {}, len(X_test) {}, len(y_train) {}, len(y_test) {}".format(len(X_train),len(X_test),len(y_train),len(y_test)))
X_train = df.drop(['average_act'],axis=1)
y_train = df['average_act']

#logistic regression model

from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)



X_test_pred_labels = logmodel.predict(X_test)
X_test_pred = logmodel.predict_proba(X_test)
X_train_pred = logmodel.predict_proba(X_train)
X_test_pred = X_test_pred[:,1]
X_train_pred = X_train_pred[:,1]

from sklearn.metrics import classification_report
print(classification_report(y_test,X_test_pred_labels))

from sklearn.metrics import accuracy_score
print("--- Accuracy of the whole model ---\n Accuracy {}\n".format(accuracy_score(y_test,X_test_pred_labels)))
print("--- Coefficient significance---\n  {}\n".format(logmodel.coef_))
print("--- Miscalibration for the whole model---\n Training Calibration {}\n Test Calibration {}\n".format(format(calMisCalibration(y_train.to_numpy(), X_train_pred)) ,calMisCalibration(y_test.to_numpy(), X_test_pred) ))
print("--- Miscalibration difference for the whole model---\n Training Calibration {}\n Test Calibration {}\n".format(format(calMisCalibrationAbsDiff(y_train.to_numpy(), X_train_pred)) ,calMisCalibrationAbsDiff(y_test.to_numpy(), X_test_pred) ))

from sklearn.model_selection import cross_val_score

scores = cross_val_score(logmodel, X_train, y_train, cv=10)
print('Cross-Validation Accuracy Scores', scores)
print('Cross-Validation Accuracy Scores', np.mean(scores))

print(df[df.columns[:]].corr())

len(X_train) 845, len(X_test) 212, len(y_train) 845, len(y_test) 212
              precision    recall  f1-score   support

           0       0.77      0.92      0.84       147
           1       0.67      0.37      0.48        65

    accuracy                           0.75       212
   macro avg       0.72      0.64      0.66       212
weighted avg       0.74      0.75      0.73       212

--- Accuracy of the whole model ---
 Accuracy 0.75

--- Coefficient significance---
  [[-5.87969350e-11  2.28529807e-10  3.03618743e-11  2.28878069e-05
  -5.26087999e-10 -3.75110432e-06]]

--- Miscalibration for the whole model---
 Training Calibration 1.0901797394560016
 Test Calibration 1.0914144899091278

--- Miscalibration difference for the whole model---
 Training Calibration 0.025680323156344793
 Test Calibration 0.028028027566477865

Cross-Validation Accuracy Scores [0.75471698 0.71698113 0.76415094 0.77358491 0.73584906 0.81132075
 0.82075472 0.7047619  0.66666667 0.78095238]
Cross-Valida

# Select Data for KD Tree and also figure out the calibration for original grid

In [10]:


counter = 0

for region_id,idxs in d_regions.items():
    counter+=1
    a_label = y_train.to_numpy()
    a_preds = X_train_pred
    
    
    mem_label = []
    mem_preds = []
    for i in idxs:
        if i<len(a_preds):
            mem_label.append(a_label[i])
            mem_preds.append(a_preds[i])
    
    print("--- region_ID --- {}\n MisCalibration {} \n Micalibration difference {}\n".format(region_id, calMisCalibration(mem_label, mem_preds), calMisCalibrationAbsDiff(mem_label, mem_preds)))
    if counter==10:
        break
    #1/0
    
    
    

ERROR: the label sum is zero
--- region_ID --- 546306
 MisCalibration inf 
 Micalibration difference 0.44654117388851433

--- region_ID --- 774499
 MisCalibration 0.3180763328992209 
 Micalibration difference 0.6819236671007791

ERROR: the label sum is zero
--- region_ID --- 544303
 MisCalibration inf 
 Micalibration difference 0.25798479785040773

ERROR: the label sum is zero
--- region_ID --- 542250
 MisCalibration inf 
 Micalibration difference 0.4157161081401519

ERROR: the label sum is zero
--- region_ID --- 526877
 MisCalibration inf 
 Micalibration difference 0.2570405477995814

--- region_ID --- 548402
 MisCalibration 0.5051695790115666 
 Micalibration difference 0.24741521049421672

ERROR: the label sum is zero
--- region_ID --- 547380
 MisCalibration inf 
 Micalibration difference 0.2432652365020705

--- region_ID --- 716102
 MisCalibration 0.3540044357523965 
 Micalibration difference 0.6459955642476035

--- region_ID --- 715077
 MisCalibration 0.5944339895396455 
 Micalibra

<ipython-input-6-83d80194f2c3>:9: RuntimeWarning: divide by zero encountered in double_scalars
  return _preds/_labels


# Fair KD-tree

# The following applies KDTree for various heights and displays results

In [9]:
#all you need to figure out is the indexes for the area




def runModel(dfc, d_new_regions, X_train, X_test,y_train, y_test):
    dfc = dfc[['rate_unemployment','percent_college','percent_married','median_income','average_act', 'percent_lunch','gridID']]
    for region_id,idxs in d_new_regions.items():
        for i in idxs:

            dfc.loc[i,'gridID']=region_id


    X_test.loc[:,'gridID'] = dfc.loc[X_test.index,'gridID']
    X_train.loc[:,'gridID'] = dfc.loc[X_train.index,'gridID']

    
    logmodel = LogisticRegression()
    logmodel.fit(X_train,y_train)
    X_test_pred_labels = logmodel.predict(X_test)
    X_test_pred = logmodel.predict_proba(X_test)
    X_train_pred = logmodel.predict_proba(X_train)
    X_test_pred = X_test_pred[:,1]
    X_train_pred = X_train_pred[:,1]

    from sklearn.metrics import classification_report
    print(classification_report(y_test,X_test_pred_labels))

    from sklearn.metrics import accuracy_score
    print("--- Accuracy of the whole model ---\n Accuracy {}\n".format(accuracy_score(y_test,X_test_pred_labels)))
    
    mem_accuracy.append(accuracy_score(y_test,X_test_pred_labels))
    
    print("--- Coefficient significance---\n  {}\n".format(logmodel.coef_))
    mem_coeff.append(logmodel.coef_)

    print("--- Miscalibration for the whole model---\n Training Calibration {}\n Test Calibration {}\n".format(format(calMisCalibration(y_train.to_numpy(), X_train_pred)) ,calMisCalibration(y_test.to_numpy(), X_test_pred) ))
    mem_train_calib.append(calMisCalibration(y_train.to_numpy(), X_train_pred))
    mem_test_calib.append(calMisCalibration(y_test.to_numpy(), X_test_pred))
    
    print("--- Miscalibration difference for the whole model---\n Training Calibration {}\n Test Calibration {}\n".format(format(calMisCalibrationAbsDiff(y_train.to_numpy(), X_train_pred)) ,calMisCalibrationAbsDiff(y_test.to_numpy(), X_test_pred) ))
    mem_train_calib_diff.append(calMisCalibrationAbsDiff(y_train.to_numpy(), X_train_pred))
    mem_test_calib_diff.append(calMisCalibrationAbsDiff(y_test.to_numpy(), X_test_pred) )
    return X_train_pred













"""
Return calibration for a given neighborhoods
"""

def returnCalibForNeighborhoods(dfc, neighborhoods, X_train_pred, y_train):

    """
    Now that we have new neighborhoods, we come up with the d_new_regions dictionary 
    """
    #print(neighborhoods)
    #1/0
    d_new_regions = defaultdict(list)
    counter = 0
    for x_start,x_end, y_start,y_end in neighborhoods:

        df_mem = dfc[ (dfc['gridX']>=x_start) & (dfc['gridX']<x_end) & (dfc['gridY']>=y_start) & (dfc['gridY']<y_end)   ]
        #print(" tricky part len(df ) {}\n ".format(len(df_mem)))
        if len(df_mem)>0:
            

            d_new_regions[counter] = df_mem.index.to_numpy()
        #d_new_regions[counter] = df_mem.index.to_numpy()
        counter+=1
    d_new_regions.keys()

    """
    Now lets calculate calibration for each neighboorhood
    """


    counter = 0
    res_holder = {}
    mem = []
    for region_id,idxs in d_new_regions.items():
        counter+=1
        a_label = y_train.to_numpy()
        a_preds = X_train_pred


        mem_label = []
        mem_preds = []
        for i in idxs:
            if i<len(a_preds):
                mem_label.append(a_label[i])
                mem_preds.append(a_preds[i])

        ################################################ you changed here 
        
        mem.append( (calMisCalibrationAbsDiff(mem_label, mem_preds)/len(mem_label), len(mem_label)) )

        #print("--- region_ID --- {}\n MisCalibration {}".format(region_id, calMisCalibrationAbsDiff(mem_label, mem_preds)))
        #if counter==10:
        #    break
    #res_holder[height] = mem

    return mem, d_new_regions








def KDTree(dfc, x_start,x_end, y_start,y_end, num_iters, counter, division_type, num_samples):
    
    
    """
    Here you have your stop conditions
    
    division_type: 
        (I)   medians
        (II)  Exhaustive Search
        (III) Sampling
        (IV)  Fairness Objective Functions
    """
    #print("counter {}".format(counter))
    if counter>=num_iters:

        temp = dfc[ (dfc['gridX']>=x_start) & (dfc['gridX']<x_end) & (dfc['gridY']>=y_start) & (dfc['gridY']<y_end)   ]
        #print("attached temp length {} neighborhood {}".format(len(temp),[x_start,x_end, y_start,y_end]))

        neighborhoods.append([x_start,x_end, y_start,y_end])
        
        return 
    

    
    counter+=1
    #print("counter {}".format(counter))
    
    
    if counter%2==0: # division is on x-axis

        if division_type=="medians":
            
            
            tot = len(dfc[(dfc['gridX']>=x_start)&(dfc['gridX']<x_end)& (dfc['gridY']>=y_start) & (dfc['gridY']<y_end)])
            tot = tot//2
            for i in range(x_start,x_end+1):
                mem_left = dfc[(dfc['gridX']>=x_start)&(dfc['gridX']<i)& (dfc['gridY']>=y_start) & (dfc['gridY']<y_end)] 
                if len(mem_left)>= tot:           
                    division_indexes = [i]          
                    break
                           
                           
                           
            
        elif division_type=="Exhaustive Search":
            
            
            mem = []
            mem_indexes = []
            #print("hi")
            for idx in range(x_start,x_end):  
                mem_left = dfc[(dfc['gridX']>=x_start)&(dfc['gridX']<idx)& (dfc['gridY']>=y_start) & (dfc['gridY']<y_end) ]
                mem_left = mem_left.index.to_numpy()
                left_miscalib = calMisCalibrationForGivenIndexesInDF(y_train.to_numpy(), X_train_pred, mem_left) 
            
                mem_right = dfc[(dfc['gridX']>=idx)&(dfc['gridX']<x_end)& (dfc['gridY']>=y_start) & (dfc['gridY']<y_end) ]
                mem_right = mem_right.index.to_numpy()       
                right_miscalib = calMisCalibrationForGivenIndexesInDF(y_train.to_numpy(), X_train_pred, mem_right) 
                #mem.append(right_miscalib+left_miscalib )

                #if len(mem_right)>20 and len(mem_left)>20:
                if True:
                    #mem.append(right_miscalib+left_miscalib)
                    mem.append(np.abs(right_miscalib-left_miscalib))
                    mem_indexes.append(idx)
            
            if len(mem_indexes)==0:
                temp = dfc[ (dfc['gridX']>=x_start) & (dfc['gridX']<x_end) & (dfc['gridY']>=y_start) & (dfc['gridY']<y_end)   ]
                #print("attached temp length {} neighborhood {}".format(len(temp),[x_start,x_end, y_start,y_end]))
                neighborhoods.append([x_start,x_end, y_start,y_end])
                return                     

            
            _min = min(mem)
            mem2 = []
            for index,i in enumerate(mem):
                if i==_min:
                    mem2.append(mem_indexes[index])
                
            f = mem2[len(mem2)//2]
            #print(" Division on X axis - x_start  {} , x_end {} , y_start {} , y_end {} Index {} \n".format(x_start,x_end, y_start,y_end,f))
            division_indexes = [f]
            
    
    
    
    
    
        
        elif division_type=="Sampling":
            pass
        elif division_type=="Fairness Objective Functions":
            pass 
        
        
        for idx in division_indexes:
            
            mem_left = dfc[(dfc['gridX']>=x_start)&(dfc['gridX']<idx)]
            mem_left = mem_left.index.to_numpy()
            left_miscalib = calMisCalibrationForGivenIndexesInDF(y_train.to_numpy(), X_train_pred, mem_left) 
            
            mem_right = dfc[(dfc['gridX']>=idx)&(dfc['gridX']<x_end)]
            mem_right = mem_right.index.to_numpy()       
            right_miscalib = calMisCalibrationForGivenIndexesInDF(y_train.to_numpy(), X_train_pred, mem_right) 
            
            #print("left calibration {} right calibration {}".format(left_miscalib, right_miscalib))
        
        
        """
        This needs to be modified
        """
        #Now you should select the best index and then 
        best_index = division_indexes[0]
        KDTree(dfc, x_start,best_index, y_start,y_end, num_iters, counter, division_type, num_samples)
        KDTree(dfc, best_index,x_end, y_start,y_end, num_iters, counter, division_type, num_samples)
        
########################################################################################### 
    else:    # division is on y-axis
        #print("yess")
        if division_type=="medians":
           # division_indexes = [y_start + (y_end - y_start)//2]
                               
                               
                               
                               
            tot = len(dfc[(dfc['gridX']>=x_start)&(dfc['gridX']<x_end)& (dfc['gridY']>=y_start) & (dfc['gridY']<y_end)])
            tot = tot//2
            for i in range(y_start,y_end+1):
                mem_left = dfc[ (dfc['gridX']>=x_start) & (dfc['gridX']<x_end) & (dfc['gridY']>=y_start)&(dfc['gridY']<i)]
                if len(mem_left)>= tot:
                    division_indexes = [i]          
                    break
                               
                               
                               
                               
                               
                               
                               
                               
        elif division_type=="Exhaustive Search":
            
            mem = []
            mem_indexes =  []
            for idx in range(y_start,y_end):  
                mem_left = dfc[ (dfc['gridX']>=x_start) & (dfc['gridX']<x_end) & (dfc['gridY']>=y_start)&(dfc['gridY']<idx)]
                mem_left = mem_left.index.to_numpy()
                left_miscalib = calMisCalibrationForGivenIndexesInDF(y_train.to_numpy(), X_train_pred, mem_left) 
            
                mem_right = dfc[ (dfc['gridX']>=x_start) & (dfc['gridX']<x_end) & (dfc['gridY']>=idx)&(dfc['gridY']<y_end)]
                mem_right = mem_right.index.to_numpy()       
                right_miscalib = calMisCalibrationForGivenIndexesInDF(y_train.to_numpy(), X_train_pred, mem_right) 
                
                #df_mem = dfc[ (dfc['gridX']>=x_start) & (dfc['gridX']<x_end) & (dfc['gridY']>=y_start) & (dfc['gridY']<y_end)   ]
                

                #if len(mem_right)>20 and len(mem_left)>20:
                if True:
                    
                    #mem.append(right_miscalib+left_miscalib)
                    mem.append(np.abs(right_miscalib-left_miscalib))
                    mem_indexes.append(idx)

            #print("len(mem_indexes)")        
            if len(mem_indexes)==0:
                temp = dfc[ (dfc['gridX']>=x_start) & (dfc['gridX']<x_end) & (dfc['gridY']>=y_start) & (dfc['gridY']<y_end)   ]
                #print("attached temp length {} neighborhood {}".format(len(temp),[x_start,x_end, y_start,y_end]))
                
                neighborhoods.append([x_start,x_end, y_start,y_end])
                return                     
                    
                
            _min = min(mem)
            mem2 = []
            for index,i in enumerate(mem):
                if i==_min:
                    mem2.append(mem_indexes[index])
                
            f = mem2[len(mem2)//2]
            #print(" Division on Y axis - x_start  {} , x_end {} , y_start {} , y_end {} Index {} \n".format(x_start,x_end, y_start,y_end,f))
            division_indexes = [f]
            
            
        elif division_type=="Sampling":
            pass
        elif division_type=="Fairness Objective Functions":
            pass 
        
        
        
        for idx in division_indexes:
            
            mem_left = dfc[(dfc['gridY']>=y_start)&(dfc['gridY']<idx)]
            mem_left = mem_left.index.to_numpy()
            left_miscalib = calMisCalibrationForGivenIndexesInDF(y_train.to_numpy(), X_train_pred, mem_left) 
            
            mem_right = dfc[(dfc['gridY']>=idx)&(dfc['gridY']<y_end)]
            mem_right = mem_right.index.to_numpy()       
            right_miscalib = calMisCalibrationForGivenIndexesInDF(y_train.to_numpy(), X_train_pred, mem_right) 
            
            #print("left calibration {} right calibration {}".format(left_miscalib, right_miscalib))
        
        
        """
        This needs to be modified
        """
        #Now you should select the best index and then 
        best_index = division_indexes[0]
        KDTree(dfc, x_start,x_end, y_start,best_index, num_iters, counter, division_type, num_samples)
        KDTree(dfc, x_start,x_end, best_index ,y_end, num_iters, counter, division_type, num_samples)
        
        
        
    
  

"""
let us define our memories
"""
mem_accuracy, mem_train_calib, mem_test_calib, mem_train_calib_diff, mem_test_calib_diff = [],[],[],[],[]
mem_num_part,mem_ave_calib,mem_ave_weighted_calib,mem_ave_weighted_calib_firstModel = [],[],[],[]
mem_coeff = []




import time

time1 = time.time()



"""
Let us start by defining 
"""

res_holder = {}

#for height in range(1,3):
for height in [10]:
#for height in range(1,11):    
    print("The tree height is {}".format(height))

    """
    APPLY KDTREE

    When we set num_iters=2k it means k division in x-axis and k division in y-axis
    division_type: 
        (I)   medians
        (II)  Exhaustive Search
        (III) Sampling
        (IV)  Fairness Objective Functions
    
    
    The output of KDTree is Neighborhoods
    """
    neighborhoods = []
    KDTree(df_kdTree, 
           x_start = 0,
           x_end = grid_size, 
           y_start = 0, 
           y_end = grid_size, 
           num_iters = height, 
           counter = 0, 
           #division_type = "Exhaustive Search", 
           division_type = "medians", 
           num_samples = 5)
    
    
    print("------------------ KDtree is Done| Neighborhoods Generated ---------------")

    res_oldModel,d_new_regions = returnCalibForNeighborhoods(df_kdTree, neighborhoods, X_train_pred, y_train)

    print("res_oldModel   {}".format(res_oldModel))
    #1/0
    print("------------------ results for the updated neighborhoods ---------------")

    

    _sum = 0
    _sum_weighted = 0
    counter=0
    counter_weighted =0 
    for value in res_oldModel:
        #print("Height: {},  Values {} \n \n".format(height,value))



        #for i in range(len(value)):
        if value[0]>0:
            _sum+= value[0]
            _sum_weighted += value[0]*value[1]

            counter_weighted += value[1]
            counter+=1

    print("Partitioning ,  Average Calib {}, Weight Calib Average {}".format(_sum/counter, _sum_weighted/counter_weighted))


    mem_ave_weighted_calib_firstModel.append( _sum_weighted/counter_weighted)
        
        
    #1/0
        
        
        
    print("------------------ Run New Model ---------------")
    
    X_train_pred_updated = runModel(df_kdTree, d_new_regions, X_train, X_test, y_train, y_test)
    res_newModel,d_new_regions = returnCalibForNeighborhoods(df_kdTree, neighborhoods, X_train_pred_updated, y_train)
    print("res_newModel   {}".format(res_newModel))
    print("------------------ results for the updated neighborhoods NewModel---------------")
 
    print(" Number of Partition {}".format(len(res_newModel)))
    mem_num_part.append(len(res_newModel))

    _sum = 0
    _sum_weighted = 0
    counter=0
    counter_weighted =0 

    
    
    for value in res_newModel:
        #print("Height: {},  Values {} \n \n".format(height,value))



        #for i in range(len(value)):
        if value[0]>0:
            _sum+= value[0]
            _sum_weighted += value[0]*value[1]

            counter_weighted += value[1]
            counter+=1


    print("Updated Model,  Average Calib {}, Weight Calib Average {}".format(_sum/counter, (_sum_weighted/counter_weighted)))

    
    
    
    
    mem_ave_calib.append(_sum/counter)
    mem_ave_weighted_calib.append( _sum_weighted/counter_weighted)
    
    
    
    
    
    time2 = time.time()
    # Seconds between Date 1 and date 2
    seconds = time2 - time1
    print("Seconds between date 1 and date 2 is % f seconds" % seconds)
    
    
    
    
    
    
    
   
    
    
    print("--------------------End -----------------------------------------------------")
    #print(neighborhoods)

The tree height is 10
------------------ KDtree is Done| Neighborhoods Generated ---------------
res_oldModel   [(0.06040224017417853, 4), (0.005013179867988393, 5), (0.5223734625211411, 1), (0.15941815497818057, 2), (0.3639263375326226, 1), (0.43605962126648445, 1), (0.15853712537255657, 2), (0.5988139913416679, 1), (0.0026294821757152365, 2), (0.07756747080964227, 4), (0.11205484236307316, 2), (0.05814187162362817, 6), (0.14293106067719047, 2), (0.06885179283626626, 3), (0.14273319838061566, 3), (0.4659360626805882, 1), (0.10767019770103131, 2), (0.07756925384236812, 3), (0.035329767793964885, 6), (0.00630747603148276, 38), (0.18469855641527705, 2), (0.2180236652247499, 1), (0.1255773465553185, 2), (0.2275690596674695, 1), (0.3434297144553668, 1), (0.3807218900904067, 1), (0.2036984541580783, 2), (0.15051868307239444, 2), (0.08803828709656729, 3), (0.14646723577498913, 2), (0.1464012932628788, 2), (0.05252159939865203, 4), (0.5038915109074716, 1), (0.2853958306839012, 1), (0.40267864

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


              precision    recall  f1-score   support

           0       0.70      0.88      0.78       147
           1       0.37      0.15      0.22        65

    accuracy                           0.66       212
   macro avg       0.54      0.52      0.50       212
weighted avg       0.60      0.66      0.61       212

--- Accuracy of the whole model ---
 Accuracy 0.660377358490566

--- Coefficient significance---
  [[-7.08365426e-07 -2.74282817e-07 -1.55487593e-06  3.08331667e-06
  -4.20177137e-06 -1.53598034e-03]]

--- Miscalibration for the whole model---
 Training Calibration 1.279988245286233
 Test Calibration 1.1836134606953292

--- Miscalibration difference for the whole model---
 Training Calibration 0.0797317519689273
 Test Calibration 0.05629657993017169

res_newModel   [(0.09791940935819829, 4), (0.029285508108064812, 5), (0.43870298392551643, 1), (0.26929779583410673, 2), (0.54069502191105, 1), (0.4294966575560767, 1), (0.26801643388011653, 2), (0.45157350938430807, 1

In [45]:

res_FairKDtree = [mem_accuracy, 
                  mem_train_calib, 
                  mem_test_calib, 
                  mem_train_calib_diff, 
                  mem_test_calib_diff,
                  mem_num_part,
                  mem_ave_calib,
                  mem_ave_weighted_calib,
                  mem_ave_weighted_calib_firstModel,
                  mem_coeff]
import pickle 

with open('res_FairKDtree_CA', 'wb') as f: 
#with open('res_FairKDtree_TX', 'wb') as f: 
    pickle.dump(res_FairKDtree, f) 

# Second Approach

In [21]:
df_kdTree.head()

,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch,gridID,gridY,gridX
0,0.122475,0.605338,0.691299,80156.0,21.568987,0.208716,546306,533,514
1,0.078767,0.764101,0.659389,70074.0,23.693529,0.094731,774499,756,355
2,0.080710,0.855984,0.953930,113229.0,22.350427,0.133333,774499,756,355
3,0.193734,0.489950,0.454356,28975.0,19.737485,0.525169,544303,531,559
4,0.212134,0.408170,0.658747,55332.0,18.663004,0.621418,544303,531,559


In [10]:


def Xsplit(dfc, x_start,x_end, y_start,y_end, f_X_train_pred, f_y_train):

            
    mem = []
    mem_indexes = []
    #print("hi")
    for idx in range(x_start,x_end):  
        mem_left = dfc[(dfc['gridX']>=x_start)&(dfc['gridX']<idx)& (dfc['gridY']>=y_start) & (dfc['gridY']<y_end) ]
        mem_left = mem_left.index.to_numpy()
        left_miscalib = calMisCalibrationForGivenIndexesInDF(f_y_train.to_numpy(), f_X_train_pred, mem_left) 

        mem_right = dfc[(dfc['gridX']>=idx)&(dfc['gridX']<x_end)& (dfc['gridY']>=y_start) & (dfc['gridY']<y_end) ]
        mem_right = mem_right.index.to_numpy()       
        right_miscalib = calMisCalibrationForGivenIndexesInDF(f_y_train.to_numpy(), f_X_train_pred, mem_right) 
        #mem.append(right_miscalib+left_miscalib )

        if len(mem_right)>20 and len(mem_left)>20:
            #mem.append(right_miscalib+left_miscalib)
            mem.append(np.abs(right_miscalib-left_miscalib))
            mem_indexes.append(idx)

    if len(mem_indexes)==0:
        temp = dfc[ (dfc['gridX']>=x_start) & (dfc['gridX']<x_end) & (dfc['gridY']>=y_start) & (dfc['gridY']<y_end)   ]
        #print("attached temp length {} neighborhood {}".format(len(temp),[x_start,x_end, y_start,y_end]))
        neighborhoods.append([x_start,x_end, y_start,y_end])
        return                     


    _min = min(mem)
    mem2 = []
    for index,i in enumerate(mem):
        if i==_min:
            mem2.append(mem_indexes[index])

    f = mem2[len(mem2)//2]
    #print(" Division on X axis - x_start  {} , x_end {} , y_start {} , y_end {} Index {} \n".format(x_start,x_end, y_start,y_end,f))
    division_indexes = [f]

    return division_indexes


def Ysplit(dfc, x_start,x_end, y_start,y_end, f_X_train_pred, f_y_train):
    
    mem = []
    mem_indexes =  []
    for idx in range(y_start,y_end):  
        mem_left = dfc[ (dfc['gridX']>=x_start) & (dfc['gridX']<x_end) & (dfc['gridY']>=y_start)&(dfc['gridY']<idx)]
        mem_left = mem_left.index.to_numpy()
        left_miscalib = calMisCalibrationForGivenIndexesInDF(f_y_train.to_numpy(), f_X_train_pred, mem_left) 

        mem_right = dfc[ (dfc['gridX']>=x_start) & (dfc['gridX']<x_end) & (dfc['gridY']>=idx)&(dfc['gridY']<y_end)]
        mem_right = mem_right.index.to_numpy()       
        right_miscalib = calMisCalibrationForGivenIndexesInDF(f_y_train.to_numpy(), f_X_train_pred, mem_right) 

        #df_mem = dfc[ (dfc['gridX']>=x_start) & (dfc['gridX']<x_end) & (dfc['gridY']>=y_start) & (dfc['gridY']<y_end)   ]


        if len(mem_right)>20 and len(mem_left)>20:


            #mem.append(right_miscalib+left_miscalib)
            mem.append(np.abs(right_miscalib-left_miscalib))
            mem_indexes.append(idx)

    #print("len(mem_indexes)")        
    if len(mem_indexes)==0:
        temp = dfc[ (dfc['gridX']>=x_start) & (dfc['gridX']<x_end) & (dfc['gridY']>=y_start) & (dfc['gridY']<y_end)   ]
        #print("attached temp length {} neighborhood {}".format(len(temp),[x_start,x_end, y_start,y_end]))

        neighborhoods.append([x_start,x_end, y_start,y_end])
        return                     


    _min = min(mem)
    mem2 = []
    for index,i in enumerate(mem):
        if i==_min:
            mem2.append(mem_indexes[index])

    f = mem2[len(mem2)//2]
    #print(" Division on Y axis - x_start  {} , x_end {} , y_start {} , y_end {} Index {} \n".format(x_start,x_end, y_start,y_end,f))
    division_indexes = [f]

    return division_indexes
    
    

    
def updatePredictions():
    
    
    
    pass

def TopDown(dfc, temp_neighborhoods, h, initial_X_train_pred, X_train, X_test, y_train, y_test):
    """
    This is the part that you need to update. basically in each step we are going to update the likelihoods.
    """
    
    
    
    #res_oldModel,d_new_regions = returnCalibForNeighborhoods(dfc, temp_neighborhoods, X_train_pred, y_train)
    #f_X_train_pred = runModel(df_kdTree, temp_neighborhoods, X_train, X_test, y_train, y_test)
    #f_y_train       = y_train

    #print("counter {}".format(counter))
    
    
        #X_train_pred_updated = runModel(df_kdTree, d_new_regions, X_train, X_test, y_train, y_test)
        #res_newModel,d_new_regions = returnCalibForNeighborhoods(df_kdTree, neighborhoods, X_train_pred_updated, y_train)

    
    f_X_train_pred   = initial_X_train_pred
    f_y_train      = y_train
    acc_mem = []
    
    for counter in range(1,h):
        print("START")
        #f_X_train_pred = runModel(dfc, temp_neighborhoods, X_train, X_test, y_train, y_test)
        #f_y_train      = y_train
        updated_temp_neighborhoods  = []
        if counter%2==0: # division is on x-axis
                
                while temp_neighborhoods:
                    x_start,x_end, y_start,y_end = temp_neighborhoods.pop(0)
                    division_indexes = Xsplit(dfc, x_start,x_end, y_start,y_end, f_X_train_pred, f_y_train)
                    
                    
                    if division_indexes:
                        best_index = division_indexes[0]
                        updated_temp_neighborhoods.append([x_start,best_index, y_start,y_end])
                        updated_temp_neighborhoods.append([ best_index,x_end, y_start,y_end])
                        
                    else:
                        updated_temp_neighborhoods.append([x_start,x_end, y_start,y_end])




        else:
                while temp_neighborhoods:
                    x_start,x_end, y_start,y_end = temp_neighborhoods.pop(0)
                    division_indexes = Ysplit(dfc, x_start,x_end, y_start,y_end, f_X_train_pred, f_y_train)
  

                    if division_indexes:
                        best_index = division_indexes[0]
                        updated_temp_neighborhoods.append([ x_start, x_end, y_start, best_index])
                        updated_temp_neighborhoods.append([ x_start, x_end, best_index, y_end])
                    
                    else:
                        updated_temp_neighborhoods.append([ x_start,x_end, y_start,y_end])
    
        temp_neighborhoods = updated_temp_neighborhoods
        
        
        
        
        #This is new#
        """
        Here temp_neighborhoods is your new neighborhoods.
        f_X_train_pred Comes from initial run for the first time 
        
        
        
        Then, next a new model is ran based on d_new_regions.
        """
        print(" The height is {} \n Neighbors are {} \n".format(counter, temp_neighborhoods))
        res_oldModel,d_new_regions = returnCalibForNeighborhoods(dfc, temp_neighborhoods, f_X_train_pred, y_train)
        f_X_train_pred = runModel(dfc, d_new_regions, X_train, X_test, y_train, y_test)
        
        #print(len(d_new_regions))
        #print(res_oldModel)
        #1/0
                  
            
            
            
            
        print("temp_neighborhoods  {}".format(temp_neighborhoods))

    

        print("res_newModel   {}".format(res_oldModel))
        print("------------------ results for the updated neighborhoods NewModel---------------")

        print(" Number of Partition {}".format(len(res_oldModel)))
        mem_num_part.append(len(res_newModel))
        
        
        _sum = 0
        _sum_weighted = 0
        counter=0
        counter_weighted =0 
        for value in res_oldModel:
            #print("Height: {},  Values {} \n \n".format(height,value))



            #for i in range(len(value)):
            if value[0]>0:
                _sum+= value[0]
                _sum_weighted += value[0]*value[1]

                counter_weighted += value[1]
                counter+=1


        print("Updated Model,  Average Calib {}, Weight Calib Average {}".format(_sum/counter, _sum_weighted/counter_weighted))

        
        mem_ave_calib.append(_sum/counter)
        mem_ave_weighted_calib.append( _sum_weighted/counter_weighted)
        
        print("--------------------End -----------------------------------------------------")
          
            
            
            
        #print(" The height is {} \n Neighbors are {} \n".format(counter, temp_neighborhoods))
    return temp_neighborhoods,res_oldModel





mem_accuracy, mem_train_calib, mem_test_calib, mem_train_calib_diff, mem_test_calib_diff = [],[],[],[],[]
mem_num_part,mem_ave_calib,mem_ave_weighted_calib,mem_ave_weighted_calib_firstModel = [],[],[],[]
mem_coeff = []


import time

time1 = time.time()

#just run it with this one because otherwise you are gonna record them multiple times.
for i in [10]:
    print("We are in the maing body and set the height  {} ".format(i))
    neighborhoods = [[0,grid_size,0,grid_size]]


    temp_neighborhoods,res_newModel = TopDown(dfc = df_kdTree, 
                                            temp_neighborhoods = neighborhoods, 
                                            h = i+1,
                                            initial_X_train_pred = X_train_pred,
                                            X_train = X_train, 
                                            X_test = X_test, 
                                            y_train = y_train, 
                                            y_test = y_test)




    




time2 = time.time()

# Seconds between Date 1 and date 2
seconds = time2 - time1
print("Seconds between date 1 and date 2 is % f seconds" % seconds)



We are in the maing body and set the height  10 
START
 The height is 1 
 Neighbors are [[0, 1024, 0, 640], [0, 1024, 640, 1024]] 



C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.69      1.00      0.82       147
           1       0.00      0.00      0.00        65

    accuracy                           0.69       212
   macro avg       0.35      0.50      0.41       212
weighted avg       0.48      0.69      0.57       212

--- Accuracy of the whole model ---
 Accuracy 0.6933962264150944

--- Coefficient significance---
  [[-5.30527908e-10 -2.64655672e-10 -1.20942511e-09 -6.95785145e-06
  -3.12002923e-09 -2.96549925e-10]]

--- Miscalibration for the whole model---
 Training Calibration 1.3930799107220089
 Test Calibration 1.270483389866294

--- Miscalibration difference for the whole model---
 Training Calibration 0.11193666331818793
 Test Calibration 0.08293122802504298

temp_neighborhoods  [[0, 1024, 0, 640], [0, 1024, 640, 1024]]
res_newModel   [(0.00020593353153880835, 633), (0.0003080010637821362, 424)]
------------------ results for the updated neighborhoods NewModel------------

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.69      1.00      0.82       147
           1       0.00      0.00      0.00        65

    accuracy                           0.69       212
   macro avg       0.35      0.50      0.41       212
weighted avg       0.48      0.69      0.57       212

--- Accuracy of the whole model ---
 Accuracy 0.6933962264150944

--- Coefficient significance---
  [[-5.32514398e-10 -2.65433580e-10 -1.21378693e-09 -6.95796769e-06
  -3.13179212e-09 -3.07823103e-09]]

--- Miscalibration for the whole model---
 Training Calibration 1.3930741454124405
 Test Calibration 1.2704776956169161

--- Miscalibration difference for the whole model---
 Training Calibration 0.11193502154129101
 Test Calibration 0.082929482146696

temp_neighborhoods  [[0, 486, 0, 640], [486, 1024, 0, 640], [0, 305, 640, 1024], [305, 1024, 640, 1024]]
res_newModel   [(0.0019233673086030169, 88), (0.00030953023281574154, 545), (0.000543269561976606, 80), (0.00012

...

              precision    recall  f1-score   support

           0       0.73      0.97      0.83       147
           1       0.75      0.18      0.30        65

    accuracy                           0.73       212
   macro avg       0.74      0.58      0.57       212
weighted avg       0.74      0.73      0.67       212

--- Accuracy of the whole model ---
 Accuracy 0.7311320754716981

--- Coefficient significance---
  [[-7.98907149e-04 -3.12461914e-04 -1.75389355e-03  3.02483438e-06
  -4.73083691e-03 -4.71844691e-02]]

--- Miscalibration for the whole model---
 Training Calibration 1.294267787226636
 Test Calibration 1.1901648271670382

--- Miscalibration difference for the whole model---
 Training Calibration 0.08379811159434003
 Test Calibration 0.058305253612535295

temp_neighborhoods  [[0, 486, 0, 549], [0, 467, 549, 620], [0, 467, 620, 640], [467, 486, 549, 640], [486, 559, 0, 494], [486, 559, 494, 497], [486, 512, 497, 512], [512, 534, 497, 512], [486, 534, 512, 514], [534, 

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


              precision    recall  f1-score   support

           0       0.81      0.94      0.87       147
           1       0.78      0.49      0.60        65

    accuracy                           0.80       212
   macro avg       0.79      0.72      0.74       212
weighted avg       0.80      0.80      0.79       212

--- Accuracy of the whole model ---
 Accuracy 0.8018867924528302

--- Coefficient significance---
  [[-4.28956338e-01  4.88027809e-02 -9.43194352e-01  3.70477020e-05
  -2.98298990e+00  3.36741630e-02]]

--- Miscalibration for the whole model---
 Training Calibration 0.9487124479072047
 Test Calibration 0.9771166809232972

--- Miscalibration difference for the whole model---
 Training Calibration 0.014605064503246357
 Test Calibration 0.0070161119810645545

temp_neighborhoods  [[0, 486, 0, 549], [0, 467, 549, 620], [0, 467, 620, 640], [467, 486, 549, 640], [486, 559, 0, 494], [486, 559, 494, 497], [486, 512, 497, 509], [486, 512, 509, 512], [512, 534, 497, 502], [51

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


              precision    recall  f1-score   support

           0       0.81      0.94      0.87       147
           1       0.78      0.49      0.60        65

    accuracy                           0.80       212
   macro avg       0.79      0.72      0.74       212
weighted avg       0.80      0.80      0.79       212

--- Accuracy of the whole model ---
 Accuracy 0.8018867924528302

--- Coefficient significance---
  [[-4.29318849e-01  5.41457831e-02 -9.42919620e-01  3.69658664e-05
  -2.99355610e+00  3.26374668e-02]]

--- Miscalibration for the whole model---
 Training Calibration 0.948637983183533
 Test Calibration 0.9770767157872018

--- Miscalibration difference for the whole model---
 Training Calibration 0.014626269689457495
 Test Calibration 0.00702836544260324

temp_neighborhoods  [[0, 486, 0, 549], [0, 467, 549, 620], [0, 467, 620, 640], [467, 486, 549, 640], [486, 559, 0, 494], [486, 559, 494, 497], [486, 512, 497, 509], [486, 512, 509, 512], [512, 534, 497, 502], [512, 

In [11]:
res_IterativeKDtree = [mem_accuracy, 
                  mem_train_calib, 
                  mem_test_calib, 
                  mem_train_calib_diff, 
                  mem_test_calib_diff,
                  mem_num_part,
                  mem_ave_calib,
                  mem_ave_weighted_calib,
                  mem_ave_weighted_calib_firstModel,
                  mem_coeff]
import pickle 
with open('res_IterativeKDtree_CA', 'wb') as f: 
#with open('res_IterativeKDtree_TX', 'wb') as f: 
    pickle.dump(res_IterativeKDtree, f) 

In [65]:
mem_coeff

[array([[-5.30527908e-10, -2.64655672e-10, -1.20942511e-09,
         -6.95785145e-06, -3.12002923e-09, -2.96549925e-10]]),
 array([[-5.32514398e-10, -2.65433580e-10, -1.21378693e-09,
         -6.95796769e-06, -3.13179212e-09, -3.07823103e-09]]),
 array([[-1.75091683e-02, -6.84808680e-03, -3.84390310e-02,
          6.59551060e-06, -1.03682815e-01, -2.32226528e-01]]),
 array([[-1.90005135e-10, -1.31474403e-10, -4.61854652e-10,
         -6.95810388e-06, -1.10357862e-09, -4.72496103e-09]]),
 array([[-2.21339867e-03, -8.65685747e-04, -4.85922016e-03,
          3.64421863e-06, -1.31069407e-02, -8.00877280e-02]]),
 array([[-1.01090812e-10, -9.66975653e-11, -2.66654270e-10,
         -6.95796330e-06, -5.77061362e-10, -5.27798148e-09]]),
 array([[-8.88062210e-04, -3.47330912e-04, -1.94962159e-03,
          2.79757498e-06, -5.25878198e-03, -4.88060095e-02]]),
 array([[-7.98907149e-04, -3.12461914e-04, -1.75389355e-03,
          3.02483438e-06, -4.73083691e-03, -4.71844691e-02]]),
 array([[-4.2895

In [75]:
with open('res_FairKDtree', 'rb') as a: 
    res_FairKDtree = pickle.load(a) 
    
with open('res_Medians', 'rb') as b: 
    res_Medians = pickle.load(b) 

    
with open('res_IterativeKDtree', 'rb') as c: 
    res_IterativeKDtree = pickle.load(c) 
